# Imports

In [ ]:
import os
import numpy as np 
import pandas as pd
import tensorflow as tf

# Read the TFRecord Files Into tf.data.Dataset Objects

In [ ]:
def decode(serialized_example):
    """ Parses a set of features and label from the given `serialized_example`.
        
        It is used as a map function for `dataset.map`

    Args:
        serialized_example (tf.Example): A serialized example containing the
            following features:
                – sensor_feature_0 – [int64]
                – sensor_feature_1 – [int64]
                – sensor_feature_2 – [int64]
                – sensor_feature_3 – [int64]
                – sensor_feature_4 – [int64]
                – sensor_feature_5 – [int64]
                – sensor_feature_6 – [int64]
                – sensor_feature_7 – [int64]
                – sensor_feature_8 – [int64]
                – sensor_feature_9 – [int64]
                – label_feature    – int64
        
    Returns:
        A decoded tf.data.Dataset object representing the tfrecord dataset
    """
    # Defaults are not specified since both keys are required.
    feature_dict = {
        'sensor_feature_0': tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True),
        'sensor_feature_1': tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True),
        'sensor_feature_2': tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True),
        'sensor_feature_3': tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True),
        'sensor_feature_4': tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True),
        'sensor_feature_5': tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True),
        'sensor_feature_6': tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True),
        'sensor_feature_7': tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True),
        'sensor_feature_8': tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True),
        'sensor_feature_9': tf.io.FixedLenSequenceFeature(shape=(), dtype=tf.int64, allow_missing=True),
        'label_feature': tf.io.FixedLenFeature(shape=(), dtype=tf.int64),
        }
    
  
    # Define a parser
    features = tf.io.parse_single_example(serialized_example, features=feature_dict)
        
    # Decode the data and capture the label feature
    sensors = [tf.cast(features[f"sensor_feature_{i}"], tf.int16) for i in range(10)]
    
    label = tf.cast(features["label_feature"], tf.int32)
    return sensors, label

def get_tfrecord_ds(tfrecord_dir):
    tfrecord_paths = [os.path.join(tfrecord_dir, f_name) \
                      for f_name in os.listdir(tfrecord_dir) \
                      if f_name.endswith('.tfrec')]
    return tf.data.TFRecordDataset(tfrecord_paths)

train_ds = get_tfrecord_ds("../input/ingv-volcanic-eruption-training-tfrecords/train")
val_ds = get_tfrecord_ds("../input/ingv-volcanic-eruption-training-tfrecords/val")
test_ds = get_tfrecord_ds("../input/ingv-volcanic-eruption-testing-tfrecords/test")

print("\n... TFRECORD DATASETS ...\n\t–– TRAIN DS – " \
      f"{train_ds}\n\t–– VAL DS   – {val_ds}\n\t–– TEST DS  – {test_ds}")

train_ds = train_ds.map(decode)
val_ds = val_ds.map(decode)
test_ds = test_ds.map(decode)

print("\n... DECODED DATASETS ...\n\t–– TRAIN DS – " \
      f"{train_ds}\n\t–– VAL DS   – {val_ds}\n\t–– TEST DS  – {test_ds}")

# Set Hyperparameters for Preprocessing and Training

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16
SHUFFLE_BUFFER = 160 # (2 TFRecords in advance)
N_EPOCHS = 50

# Preprocessing

Initial work will be done with no real preprocessing to set a dumb baseline

In [ ]:
def preprocessing_fn(x,y):
    def _preprocess_y(_y, max_to_norm=50000000):
        """ Approximate reduction from millions to integers """
        return tf.divide(tf.cast(_y, tf.float32), tf.cast(max_to_norm, tf.float32))
    
    def _preprocess_x(_x, max_to_norm=1000):
        """ No active normalization """
        return tf.divide(tf.cast(tf.transpose(_x), tf.float32), tf.cast(max_to_norm, tf.float32))
    
    x = _preprocess_x(x)
    y = _preprocess_y(y)
    return x,y 
    
pp_train_ds = train_ds.shuffle(SHUFFLE_BUFFER)
pp_train_ds = pp_train_ds.map(lambda x,y: preprocessing_fn(x,y),
                              num_parallel_calls=AUTOTUNE)
pp_train_ds = pp_train_ds.batch(BATCH_SIZE)
final_train_ds = pp_train_ds.prefetch(AUTOTUNE)

pp_val_ds = val_ds.map(lambda x,y: preprocessing_fn(x,y),
                       num_parallel_calls=AUTOTUNE)
pp_val_ds = pp_val_ds.batch(BATCH_SIZE)
final_val_ds = pp_val_ds.prefetch(AUTOTUNE)

pp_test_ds = test_ds.map(lambda x,y: (tf.divide(tf.cast(tf.transpose(x), tf.float32), tf.cast(1000, tf.float32)), y))
pp_test_ds = pp_test_ds.batch(BATCH_SIZE)
final_test_ds = pp_test_ds.prefetch(AUTOTUNE)

print("\n... PREPROCESSED DATASETS ...\n\t–– TRAIN DS – " \
      f"{final_train_ds}\n\t–– VAL DS   – {final_val_ds}\n\t–– TEST DS  – {final_test_ds}")

# Basic SqueezeNet Model

In [ ]:
def sqz_exp(inputs, squeeze, expand, residual, kernel_width=10, batch_norm=True):
    
    SQZ_ARGS = dict(filters=squeeze, kernel_size=1, padding='same', use_bias=not batch_norm)
    EXP_BR_1_ARGS = dict(filters=expand, kernel_size=1, padding='same', use_bias=not batch_norm)
    EXP_BR_2_ARGS = dict(filters=expand, kernel_size=kernel_width, padding='same', use_bias=not batch_norm)
    
    # Squeeze Part
    x = tf.keras.layers.Conv1D(**SQZ_ARGS)(inputs)
    if batch_norm:
        x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    # Branching Part (Expand Part)
    br_1 = tf.keras.layers.Conv1D(**EXP_BR_1_ARGS)(x)
    br_2 = tf.keras.layers.Conv1D(**EXP_BR_2_ARGS)(x)
    if batch_norm:
        br_1 = tf.keras.layers.BatchNormalization()(br_1)
        br_2 = tf.keras.layers.BatchNormalization()(br_2)
    combine = tf.keras.layers.concatenate([br_1, br_2])
    combine = tf.keras.layers.Activation("relu")(combine)
    
    # Residual Part

    if residual:
        return tf.keras.layers.Concatenate()([combine, inputs])
    else:
        return combine

# this is to make it behave similarly to other Keras layers
def sqz_module(squeeze, expand, kernel_width, residual=False, bn=True):
    return lambda x: sqz_exp(x, squeeze, expand, residual, kernel_width, bn)

def get_sqz_model(input_shape=(60001,10),
                  n_blocks=3, 
                  init_kernel_width=200,
                  init_sqz_size=32,
                  init_exp_size=128,
                  n_dense=1024,
                  pool_size=10):

    inputs = tf.keras.layers.Input(shape=input_shape)
    x = inputs

    for i in range(n_blocks):
        if i==0:
            x = sqz_module(squeeze=init_sqz_size, 
                           expand=init_exp_size, 
                           kernel_width=init_kernel_width,
                           residual=False)(x)
        else:
            x = tf.keras.layers.MaxPooling1D(max(2, pool_size-(i-1)*2))(x)
            x = sqz_module(squeeze=init_sqz_size*i, 
                expand=init_exp_size*i, 
                kernel_width=max(init_kernel_width//2**(i+1), 10),
                residual=True)(x)

    x = tf.keras.layers.Dense(n_dense, activation='relu')(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(n_dense//2, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.6)(x)
    
    # OUTPUT
    outputs = tf.keras.layers.Dense(1, activation='linear')(x)
    return tf.keras.Model(inputs=inputs, outputs=[outputs])

sqznet = get_sqz_model()
sqznet.summary()
tf.keras.utils.plot_model(sqznet, show_shapes=True)

# Compile the Model

In [ ]:
sqznet.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00225), loss=tf.keras.losses.Huber(), metrics="mae")

# Callbacks

In [ ]:
CKPT_PATH = "./ckpts/EPOCH__{epoch:04d}___MAE__{val_mae:.2f}.ckpt"
ckpt_cb = tf.keras.callbacks.ModelCheckpoint(CKPT_PATH, monitor="val_mae", verbose=1)
lr_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_mae', factor=0.5, patience=2, verbose=1, min_lr=0.00001)
early_cb = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=6, verbose=1, restore_best_weights=True)
cb_list = [ckpt_cb, lr_cb, early_cb]

# Fitting

In [ ]:
sqznet.fit(final_train_ds, validation_data=final_val_ds, 
           epochs=N_EPOCHS, 
           callbacks=cb_list)

# Prediction

In [ ]:
# sqznet = tf.keras.models.load_model("/tmp/ckpts/EPOCH__0006___MAE__6.5477.ckpt", compile=True)

time_to_eruption = np.zeros((4520,), dtype=np.int32)
segment_id = np.zeros((4520,), dtype=np.int32)

for i, (inputs, ids) in enumerate(final_test_ds):
    print(f"Infering on Batch {i+1}")
    segment_id[BATCH_SIZE*i:min(BATCH_SIZE*(i+1), 4520)] = ids.numpy()
    time_to_eruption[BATCH_SIZE*i:min(BATCH_SIZE*(i+1), 4520)] = sqznet.predict(inputs)[:, 0]*50000000

pred_df = pd.DataFrame({"segment_id":segment_id, "time_to_eruption":time_to_eruption})
pred_df.to_csv('submission.csv', index=False)

In [ ]:
pred_df

In [ ]:
pred_df.describe().T